# Roteiro para treinamento do DINOv3

Inicialmente será realizado um treinamento com uma pequena porção do dataset, contendo alguns recortes de ortofotos. Caso esse treino dê certo, estenderemos para o dataset maior.

## Objetivos
* Verificar se o DINOv3 é capaz de aprender o contexto de soja

## Objetivos específicos
* Criar um modelo especialista em imagens de plantação de soja sem anotações
* Criar uma representação visual (via PCA ou foreground segmentation), usando o modelo aprendido no treino

## Etapas:
**1) Organização do dataset**
- Objetivo: converter o dataset no formato ImageNet
- Formato esperado pelo DINO:

        dataset_soja_dinov3/
        ├── train/
        │   └── soja/ 
        │       ├── soja_train_000001.jpg
        │       ├── soja_train_000002.jpg
        │       └── ...
        ├── val/
        │   └── soja/
        │       ├── soja_val_000001.jpg
        │       └── ...
        └── extra/ -> Metadados
            ├── class-ids-TRAIN.npy
            ├── class-names-TRAIN.npy
            └── ...

**2. Organização do ambiente**
- Verificar dependências específicas do DINO
- Avaliar compatibilidade da GPU

**3) Treinamento**
- Ajustar o arquivo `yaml` para o treino
- Iniciar o treinamento


### 1) Organização do dataset
- Objetivo: converter o dataset no formato ImageNet
- Formato esperado pelo DINO:

        dataset_soja_dinov3/
        ├── train/
        │   └── soja/ 
        │       ├── soja_train_000001.jpg
        │       ├── soja_train_000002.jpg
        │       └── ...
        ├── val/
        │   └── soja/
        │       ├── soja_val_000001.jpg
        │       └── ...
        └── extra/ -> Metadados
            ├── class-ids-TRAIN.npy
            ├── class-names-TRAIN.npy
            └── ...

In [4]:
# Imports

import os
import shutil
from pathlib import Path
import random

import numpy as np

In [11]:
def organizar_dataset(source_dir, target_dir, split_ratio=0.8):
    source_path = Path(source_dir)
    target_path = Path(target_dir)
    
    train_dir = target_path / 'train' / 'soja'
    val_dir = target_path / 'val' / 'soja'
    
    train_dir.mkdir(parents=True, exist_ok=True)
    val_dir.mkdir(parents=True, exist_ok=True)
    
    image_extensions = ['*.jpg', '*.jpeg', '*.png', '*.JPG', '*.JPEG', '*.PNG']
    all_images = []
    
    for ext in image_extensions:
        all_images.extend(source_path.glob(ext))
    
    print(f"📊 Total de imagens encontradas: {len(all_images)}")
    
    random.shuffle(all_images)
    split_index = int(len(all_images) * split_ratio)
    train_images = all_images[:split_index]
    val_images = all_images[split_index:]

    for i, img in enumerate(train_images):
        new_name = f"soja_train_{i:06d}{img.suffix}"
        shutil.copy2(img, train_dir / new_name)

    for i, img in enumerate(val_images):
        new_name = f"soja_val_{i:06d}{img.suffix}"
        shutil.copy2(img, val_dir / new_name)
        
    print(f"✅ Dataset organizado:")
    print(f"  📂 Treino: {len(train_images)} imagens")
    print(f"  📂 Validação: {len(val_images)} imagens")
    print(f"  📍 Localização: {target_path}")
    
def create_metadata_files(dataset_path):
    
    # Criar diretório 'extra' para arquivos de metadados
    extra_dir = dataset_path / 'extra'
    extra_dir.mkdir(exist_ok=True)
    
    # Listar imagens de treino
    train_images = list((dataset_path / "train" / "soja").glob("*"))
    print(f"📊 Encontradas {len(train_images)} imagens de treino")
    
    # 🔧 SOLUÇÃO: Usar dtype estruturado memory-mappable
    # Definir dtype que suporta memory-mapping
    max_path_length = max(len(f"train/soja/{img.name}") for img in train_images) + 10
    
    dtype_entries = np.dtype([
        ('path', f'U{max_path_length}'),  # String Unicode de tamanho fixo
        ('class_id', np.int32)            # Inteiro 32-bit
    ])
    
    print(f"📋 Usando dtype: {dtype_entries}")
    print(f"📏 Tamanho máximo de path: {max_path_length}")
    
    # Criar array estruturado
    train_entries = np.empty(len(train_images), dtype=dtype_entries)
    
    for i, img in enumerate(train_images):
        rel_path = f"train/soja/{img.name}"
        train_entries[i] = (rel_path, 0)  # (path, class_id)
    
    # Salvar como .npy
    entries_file = extra_dir / "entries-TRAIN.npy"
    np.save(entries_file, train_entries)
    print(f"✅ Criado {entries_file}")
    print(f"   📊 Shape: {train_entries.shape}")
    print(f"   📊 Dtype: {train_entries.dtype}")
    
    # 🔧 TESTE CRÍTICO: Verificar memory-mapping
    try:
        test_mmap = np.load(entries_file, mmap_mode="r")
        print(f"✅ Memory-mapping TEST: SUCESSO!")
        print(f"   📋 Primeira entrada: {test_mmap[0]}")
        print(f"   📋 Tipos: path={type(test_mmap[0]['path'])}, class_id={type(test_mmap[0]['class_id'])}")
        
        # Verificar se consegue acessar path e class_id
        first_path = test_mmap[0]['path']
        first_class = test_mmap[0]['class_id']
        print(f"   📋 Path: '{first_path}', Class: {first_class}")
        
    except Exception as e:
        print(f"❌ ERRO no memory-mapping: {e}")
        return False
    
    # Criar arquivos auxiliares também
    class_ids = np.zeros(len(train_images), dtype=np.int32)
    class_names = np.array(['soja'] * len(train_images), dtype='U10')  # String fixa
    
    np.save(extra_dir / "class-ids-TRAIN.npy", class_ids)
    np.save(extra_dir / "class-names-TRAIN.npy", class_names)
    
    # Processar VAL se existir
    val_dir = dataset_path / "val" / "soja"
    if val_dir.exists():
        val_images = list(val_dir.glob("*"))
        print(f"📊 Encontradas {len(val_images)} imagens de validação")
        
        # VAL entries
        val_entries = np.empty(len(val_images), dtype=dtype_entries)
        for i, img in enumerate(val_images):
            rel_path = f"val/soja/{img.name}"
            val_entries[i] = (rel_path, 0)
        
        val_entries_file = extra_dir / "entries-VAL.npy"
        np.save(val_entries_file, val_entries)
        
        # Testar VAL memory-mapping também
        try:
            test_val_mmap = np.load(val_entries_file, mmap_mode="r")
            print(f"✅ VAL Memory-mapping TEST: SUCESSO!")
        except Exception as e:
            print(f"❌ ERRO VAL memory-mapping: {e}")
            return False
        
        # VAL auxiliares
        val_class_ids = np.zeros(len(val_images), dtype=np.int32)
        val_class_names = np.array(['soja'] * len(val_images), dtype='U10')
        
        np.save(extra_dir / "class-ids-VAL.npy", val_class_ids)
        np.save(extra_dir / "class-names-VAL.npy", val_class_names)
        
        print(f"✅ Todos os arquivos VAL criados")
    
    print(f"\n🎉 TODOS OS ARQUIVOS CRIADOS COM SUCESSO!")
    print(f"📁 Diretório: {extra_dir}")
    
    # Listar arquivos criados
    print("📋 Arquivos criados:")
    for file in extra_dir.glob("*.npy"):
        file_size = file.stat().st_size / 1024  # KB
        print(f"   📄 {file.name}: {file_size:.1f} KB")
    
    return True

In [10]:
source_dir = "/home/lades/computer_vision/wesley/dataset/daninhas_multiclasse2/teste-dino/"
target_dir = "/home/lades/computer_vision/wesley/dino-soja/dataset/teste-organizado-dinov3/"

organizar_dataset(source_dir, target_dir, split_ratio=0.8)

create_metadata_files(Path(target_dir))

📊 Total de imagens encontradas: 764
✅ Dataset organizado:
  📂 Treino: 611 imagens
  📂 Validação: 153 imagens
  📍 Localização: /home/lades/computer_vision/wesley/dino-soja/dataset/teste-organizado-dinov3
📊 Encontradas 611 imagens de treino
✅ Criado entries-TRAIN.npy: (611, 2)
✅ Teste carregamento: OK, shape=(611, 2)
📋 Primeira entrada: ['train/soja/soja_train_000367.jpg' 0]
📋 Tipo primeira entrada: <class 'str'>, <class 'int'>
✅ Criados todos os arquivos auxiliares
📊 Encontradas 153 imagens de validação
✅ Criados arquivos VAL também


In [12]:
create_metadata_files(Path(target_dir))

📊 Encontradas 611 imagens de treino
📋 Usando dtype: [('path', '<U42'), ('class_id', '<i4')]
📏 Tamanho máximo de path: 42
✅ Criado /home/lades/computer_vision/wesley/dino-soja/dataset/teste-organizado-dinov3/extra/entries-TRAIN.npy
   📊 Shape: (611,)
   📊 Dtype: [('path', '<U42'), ('class_id', '<i4')]
✅ Memory-mapping TEST: SUCESSO!
   📋 Primeira entrada: ('train/soja/soja_train_000367.jpg', 0)
   📋 Tipos: path=<class 'numpy.str_'>, class_id=<class 'numpy.int32'>
   📋 Path: 'train/soja/soja_train_000367.jpg', Class: 0
📊 Encontradas 153 imagens de validação
✅ VAL Memory-mapping TEST: SUCESSO!
✅ Todos os arquivos VAL criados

🎉 TODOS OS ARQUIVOS CRIADOS COM SUCESSO!
📁 Diretório: /home/lades/computer_vision/wesley/dino-soja/dataset/teste-organizado-dinov3/extra
📋 Arquivos criados:
   📄 class-ids-TRAIN.npy: 2.5 KB
   📄 class-ids-VAL.npy: 0.7 KB
   📄 class-names-TRAIN.npy: 24.0 KB
   📄 class-names-VAL.npy: 6.1 KB
   📄 entries-VAL.npy: 25.8 KB
   📄 entries-TRAIN.npy: 102.8 KB


True